## Importa bibliotecas

In [1]:
import pandas as pd
import numpy as np
import re
import pyodbc
from pprint import pprint
import matplotlib.pyplot as plt
from scipy import stats

##  Cria conexões com os bancos de dado

In [2]:
import pyodbc
bdworgs = pyodbc.connect('Driver={SQL Server};'
                      'Server=bduorgs,5468;'
                      'Database=SFG_BD;'
                      'Trusted_Connection=yes;')

skate = pyodbc.connect('Driver={SQL Server};'
                      'Server=SAG003\SBD172;'
                      'Database=FiscalizacaoGeracao;'
                      'Trusted_Connection=yes;')

## Baixando banco de dados

In [3]:
sql_bdworgs_query = """
SELECT  [Idempreendimento]
      ,[Id_UG]
      ,[Num_UG]
      ,[Pot_UG]
      ,[OpTest_Outor_UG]
      ,[OpCom_Outor_UG]
      ,[OpTest_R_UG]
      ,[OpCom_R_UG]
      ,[UF]
      ,[PotenciaOutorgada]
      ,[Leilao]
      ,[Conexao]
      ,[Sist_Trans_Outorga]
      ,[Obras_Outorga]
      ,[Concretagem_Outorga]
      ,[Montagem_Outorga]
      ,[Enchimento_Outorga]
      ,[Torres_Outorga]
      ,[Sist_Trans_Real]
      ,[Obras_Real]
      ,[Concretagem_Real]
      ,[Montagem_Real]
      ,[Enchimento_Real]
      ,[Torres_Real]
      ,[observacao1]
      ,[observacao2]
      ,[ClasseCombustivel]
      ,[DscComercializacaoEnergia]
      ,[DscSistema]
      ,[Skate]
      ,[FonteDataOC]
  FROM [SFG_BD].[SK8].[DadosHistoricos2020]
"""
bdworgs = pd.read_sql_query(sql_bdworgs_query,bdworgs)
bdworgs.to_feather("bdworgs.feather")

In [4]:
skate_leilao_query = """
SELECT [IdeUsinaOutorga]
      ,[CodCegFormatado]
      ,[SigTipoGeracao]
      ,[NomUsina]
      ,[CodLeilao]
      ,[DatInicioSuprimento]
      ,[VlrGFContratadoMW]
      ,[VlrPotenciaResultadoMW]
      ,[VlrPrecoVenda]
  FROM [FiscalizacaoGeracao].[SKT].[vMonitoramentoLeilao]
"""



skate_leilao = pd.read_sql_query(skate_leilao_query,skate)
skate_leilao.to_feather("skate_leilao.feather")

In [5]:
skate_ug_query = """
SELECT [IdeUsinaOutorga]
      ,[CodCegFormatado]
      ,[SigTipoGeracao]
      ,[NomUsina]
      ,[NumUgUsina]
      ,[MdaPotenciaUnitaria]
      ,[MdaPotenciaLiberadaTeste]
      ,[MdaPotenciaLiberadaComercial]
      ,[NumDespachoTeste]
      ,[numDespachoComercial]
      ,[IdcMonitorada]
      ,[DatDescidaoutorgadaRotor]
      ,[DatRealizacaoSFGRotor]
      ,[DatInicioOpTesteOutorgada]
      ,[DatLiberacaoSFGTeste]
      ,[DatUGInicioOpComerOutorgado]
      ,[DatLiberOpComerRealizado]
      ,[DatPrevisaoSFGComercial]
  FROM [FiscalizacaoGeracao].[SKT].[vMonitoramentoUG]
"""
skate_ug = pd.read_sql_query(skate_ug_query,skate)
skate_ug.to_feather("skate_ug.feather")


id_tipo_usina = pd.read_sql_query("""
SELECT [IdeAgente]
      ,[SigTipoGeracao]
  FROM [FiscalizacaoGeracao].[SKT].[vUsinaOutorgas]
""",skate)
id_tipo_usina.to_feather("id_tipo_usina.feather")


skate_usinas_query = """
SELECT [IdeUsinaOutorga]
      ,[DatOutorgaUsina]
      ,[UFUsina]
      ,[IdcDescOrigemCombustivel]
      ,[SigTipoGeracao]
      ,[CodCegFormatado]
      ,[NomUsina]
      ,[VlrPotenciaOutorgada]
      ,[VlrPotenciaFiscalizada]
      ,[DscFaseUsina]
      ,[VlrGarantiaFisica]
      ,[DscTipoOutorga]
      ,[DscAtoOutorga]
      ,[NomOrgaoOutorgante]
      ,[NumNumeroAto]
      ,[DatEmissaoAto]
      ,[DscProprietario]
      ,[DscTipoConexao]
      ,[NomConexao]
      ,[MdaTensaoConexao]
      ,[NomEmpresaConexao]
      ,[NumCNPJ]
      ,[IdcUsinaMonitorada]
      ,[IdcViabilidade]
      ,[IdcSituacaoObra]
      ,[IdcObraParalisada]
      ,[IdcSemPrevisão]
      ,[DatPrevisaoIniciobra]
      ,[DatContratoEPCOutorgado]
      ,[DatRecursoFinanceiroOutorgado]
      ,[DatCanteiroObraOutorgado]
      ,[DatCanteiroObraRealizado]
      ,[DatInicioObraOutorgado]
      ,[DatInicioObraRealizado]
      ,[DatConcretagemOutorgado]
      ,[DatConcretagemRealizado]
      ,[DatDesvioRioOutorgado]
      ,[DatDesvioRioRealizado]
      ,[DatMontagemOutorgado]
      ,[DatMontagemRealizado]
      ,[DatConclusaoTorresOutorgado]
      ,[DatConclusaoTorresRealizado]
      ,[DatEnchimentoOutorgado]
      ,[DatEnchimentoRealizado]
      ,[DatComissionamentoUGRealizado]
      ,[DatSisTransmissaoOutorgado]
      ,[DatSisTransmissaoRealizado]
      ,[DatConclusaoSisTransOutorgado]
      ,[DatConclusaoSisTransRealizado]
      ,[DscJustificativaPrevisao]
      ,[DatMonitoramento]
      ,[DscObservacaoFiscal]
      ,[NumProcessoSicnet]
      ,[DscAssuntoSicnet]
      ,[DscComercializacaoEnergia]
      ,[DscSistema]
      ,[DatConclusaoTransporteRealizado]
      ,[IdcSituacaoCronograma]
  FROM [FiscalizacaoGeracao].[SKT].[vMonitoramentoUsina]
"""
skate_usinas = pd.read_sql_query(skate_usinas_query,skate)
skate_usinas.to_feather("skate_usinas.feather")

### Corrigindo banco de dados bdworgs


In [6]:
bdworgs = pd.read_excel("./bdworgs.xlsx") # Banco de dados com correções é importado
bdworgs.Num_UG = bdworgs.Num_UG.astype(str) 
bdworgs.to_feather("./bdworgs_corrigido.feather") 

In [7]:
def corrige_ug(ug): # Função criada para corrigir nomes das UGs
    ug = ug.replace(" ","")
    ug = ug.replace("e",",")
    ug = ug.replace("-","a")
    ug = re.sub(r",+",r",",ug)
    ug = re.sub(r"([,ae])(0)",r"\1",ug)
    ug = re.sub(r"^0",r"",ug)
    ug = ug.split("a")
    lista = []
    for i in range(len(ug)-1):
        menor = int(ug[i].split(",")[-1])
        maior = int(ug[i+1].split(",")[0])
        list_nums = list(range(menor+1,maior))
        lista += list_nums
    ug = [el.split(",") for el in ug]
    ug = [int(el2) for el1 in ug for el2 in el1]
    ug+= lista
    return (sorted(ug))

### Carrega bancos de dados

In [8]:
# Carrega bdworgs
bdworgs = pd.read_feather("./bdworgs_corrigido.feather")

# Carrega três bancos de dados do skate
skate_ug = pd.read_feather("./skate_ug.feather")
skate_usinas = pd.read_feather("./skate_usinas.feather")
skate_leilao = pd.read_feather("skate_leilao.feather")

In [9]:
skate_leilao.sort_values(by="DatInicioSuprimento",inplace=True)
skate_leilao.drop_duplicates(keep="first",subset= ["IdeUsinaOutorga"],inplace=True)

skate_usinas = pd.merge(skate_usinas,
        skate_leilao[["DatInicioSuprimento","IdeUsinaOutorga"]],
        on="IdeUsinaOutorga")


mask = skate_usinas.DscComercializacaoEnergia == "ACR"
skate_usinas[["DscComercializacaoEnergia","DatInicioSuprimento"]].sample(10)

,DscComercializacaoEnergia,DatInicioSuprimento
570,Fora do ACR,None
1070,Fora do ACR,None
1331,Fora do ACR,None
1837,Fora do ACR,None
1769,Fora do ACR,None
2039,ACR,2024-01-01
1859,Fora do ACR,None
671,ACR,2025-01-01
1539,ACR,2025-01-01
796,ACR,2017-01-01


### No dataframe bdworgs são feitas as seguintes alterações:
 - Onde não não existe data na coluna DatMontagemOutorgado e existe na coluna DatConclusaoTorresOutorgado, o valor que está na coluna DatConclusaoTorresOutorgado passa para a coluna DatMontagemOutorgado.
 - Mesmo processo feito para as colunas DatConclusaoTorresOutorgado/DatMontagemOutorgado é feito para as colunas DatConclusaoTorresRealizado/DatMontagemRealizado

In [10]:
mask = bdworgs.Montagem_Outorga.isnull() 
bdworgs.loc[mask,"Montagem_Outorga"] = bdworgs.loc[mask,"Torres_Outorga"]

mask = bdworgs.Montagem_Outorga.isnull() 
bdworgs.loc[mask,"Montagem_Real"] = bdworgs.loc[mask,"Torres_Real"]

### Tabela com associações de nomes entre os bancos de dados

In [11]:
# Carrega tabela de associações
associassoes = pd.read_excel("associassoes.xlsx")
associassoes

,bdworgs,skate
0,Idempreendimento,IdeUsinaOutorga
1,OpTest_Outor_UG,DatInicioOpTesteOutorgada
2,OpCom_Outor_UG,DatUGInicioOpComerOutorgado
3,OpTest_R_UG,DatLiberacaoSFGTeste
4,OpCom_R_UG,DatLiberOpComerRealizado
5,NaN,DatOutorgaUsina
6,UF,UFUsina
7,NaN,SigTipoGeracao
8,PotenciaOutorgada,VlrPotenciaOutorgada
9,Obras_Outorga,DatInicioObraOutorgado


In [12]:
# Cria dicionário dos nomes das colunas para poder relacionar os nomes das colunas
# Dicionário de colunas bdworgs > skate
dict_bdworgs_skate = dict(zip(associassoes.dropna()["bdworgs"],associassoes.dropna()["skate"]))
# Dicionário de colunas skate > bdworgs
dict_skate_bdworgs = dict(zip(associassoes.dropna()["skate"],associassoes.dropna()["bdworgs"]))

In [13]:
# Colunas carregados do skate
skate_cols = [el for el in associassoes["skate"].dropna()]
# Colunas carregadas do bdworgs
bdworgs_cols =[el for el in associassoes["bdworgs"].dropna()]
skate_ug_cols = []
skate_usinas_col = []
for col in skate_cols:
    if col in skate_ug.columns:
        skate_ug_cols.append(col)
    if col in skate_usinas.columns:
        skate_usinas_col.append(col)
        
bdworgs = bdworgs[bdworgs_cols]
skate_ug = skate_ug[skate_ug_cols]
skate_usinas = skate_usinas[skate_usinas_col]
bdworgs = bdworgs.rename(columns=dict_bdworgs_skate)
bdworgs.sample(5)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,DatMontagemRealizado,Torres_Outorga,Torres_Real,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,DscComercializacaoEnergia,Num_UG,Pot_UG
92,30480,2013-06-22,2013-07-01,2013-11-09,2014-01-03,RS,30.00,2011-10-01,2012-08-01,2012-01-01,...,2013-09-06,2013-05-01,2013-09-15,None,None,2011-11-01,2012-05-01,ACR,1 a 3,9.0000
1310,35708,2017-01-04,2017-01-14,2016-12-31,2017-01-31,PA,8.04,2016-09-18,2016-11-15,None,...,2016-12-05,None,None,None,None,2016-11-27,2016-12-05,ACR,7,0.6216
114,30509,2015-02-28,2015-03-31,2015-07-04,2015-08-20,RN,27.00,2012-08-01,2013-12-15,2012-10-02,...,None,2014-10-15,2015-03-12,None,None,2012-06-01,2015-06-15,ACR,1 a 10,27.0000
1769,28933,None,None,2011-03-01,2011-06-15,PE,20.20,None,None,None,...,None,None,None,None,None,None,None,Fora do ACR,4,10.0000
1013,33139,2017-11-15,2018-01-01,2017-08-29,2017-09-19,PI,29.40,2017-03-15,2016-09-15,2017-05-01,...,None,2017-08-01,2017-05-26,None,None,2017-04-01,2016-10-15,ACR,1 a 3,6.3000


In [14]:
skate_usinas_col

['IdeUsinaOutorga',
 'DatOutorgaUsina',
 'UFUsina',
 'SigTipoGeracao',
 'VlrPotenciaOutorgada',
 'DatInicioObraOutorgado',
 'DatInicioObraRealizado',
 'DatConcretagemOutorgado',
 'DatConcretagemRealizado',
 'DatMontagemOutorgado',
 'DatMontagemRealizado',
 'DatEnchimentoOutorgado',
 'DatEnchimentoRealizado',
 'DatSisTransmissaoOutorgado',
 'DatSisTransmissaoRealizado',
 'DatInicioSuprimento',
 'DscComercializacaoEnergia']

### Adiciona as seguintes  colunas no bdworgs:
- Quant_UG  &rarr; Quantidade de UGs agrupadas
- SigTipoGeracao &rarr; Tipo de geração


In [15]:
# Insere coluna Quant_UG em bdworgs
bdworgs.insert(3,"Quant_UG",bdworgs["Num_UG"].apply(lambda x : len(corrige_ug(x))))
# Remove coluna Num_UG
bdworgs = bdworgs.drop("Num_UG",axis=1)

ceg_df = pd.read_excel("./Relatorio_Outorgas.xlsx")[["Identificador da Usina","Tipo de Geração"]]
dict_id_ceg = dict(zip(ceg_df["Identificador da Usina"],ceg_df["Tipo de Geração"]))
bdworgs["SigTipoGeracao"] = bdworgs["IdeUsinaOutorga"].map(dict_id_ceg)
bdworgs.sample(5)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,Quant_UG,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,...,DatMontagemRealizado,Torres_Outorga,Torres_Real,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,DscComercializacaoEnergia,Pot_UG,SigTipoGeracao
1739,28840,2005-12-06,2005-12-06,3,2006-12-20,2007-02-07,RO,25.70,None,None,...,None,None,None,None,None,None,None,Fora do ACR,13.65,PCH
1470,26792,2014-09-14,2014-10-31,1,2014-08-05,2014-09-17,PA,392.95,2011-08-01,2011-10-31,...,2012-10-30,None,None,2014-02-15,2014-05-13,2012-08-30,2012-09-12,ACR,123.33,UHE
2122,29736,2013-02-28,2013-03-31,1,2014-07-10,2014-07-29,RO,3750.00,2010-09-01,2009-12-01,...,2010-04-01,None,None,2012-11-30,2012-10-19,None,None,ACR,75.00,UHE
425,31069,2016-08-31,2016-08-31,2,2016-05-06,2016-07-27,RN,25.20,2015-07-27,2015-04-15,...,None,2016-06-22,2016-03-07,None,None,2015-07-01,2015-04-15,Fora do ACR,4.20,EOL
2171,29765,None,None,2,2008-11-06,2008-12-19,SP,80.00,None,2006-09-15,...,None,None,None,None,None,None,None,Fora do ACR,80.00,UTE


### No BD skate_ug, linhas com todas as colunas iguais tais como IdeUsinaOutorga, DatInicioOpTesteOutorgada... (exceto  MdaPotenciaUnitaria, que não necessariamente tem que ser igual em todas as linhas)  são agrupadas e são criadas duas outras colunas: 
- Quant_UG -> &rarr; Quantidade de UGs agrupadas
- Pot_UG -> &rarr; Soma da potência das UGs que foram agrupadas

#### Entrada: skate_ug
#### Saída: skate_ug_grouped

In [16]:
skate_ug_group_list = [x for x  in skate_ug.columns.to_list() if x not in ["MdaPotenciaUnitaria"]]
skate_ug_grouped = skate_ug.groupby(skate_ug_group_list,dropna=False).agg(
    Quant_UG = ("MdaPotenciaUnitaria","count"),
    Pot_UG = ("MdaPotenciaUnitaria","sum")
    
).reset_index()

### Realiza o processo acima para o bdworgs:
- Quant_UG -> &rarr; Quantidade de UGs agrupadas. Nesse caso, como o bdworgs, já possuía a coluna Quant_UG, os valores dessa coluna foram somados para os valores agrupados.
- Pot_UG -> &rarr; Soma da potência dos conjuntos de UGs que foram agrupados.

#### Entrada: bdworgs
#### Saída: bdworgs_ug_grouped

In [17]:
dummy_bdworgs = bdworgs.rename(columns={"Quant_UG":"Quant_UG_","Pot_UG":"Pot_UG_"})
dummy_bdworgs_group_list = [x for x  in dummy_bdworgs.columns.to_list() if x not in ["Quant_UG_","Pot_UG_"]]

bdworgs_ug_grouped = dummy_bdworgs.groupby(dummy_bdworgs_group_list,dropna=False).agg(
    Quant_UG = ("Quant_UG_","sum"),
    Pot_UG = ("Pot_UG_","sum")
    
).reset_index() 

### Checa se a quantidade de potência e de usinas nos dataframes iniciais e agrupados são iguais.

In [18]:
print(skate_ug_grouped.Quant_UG.sum() - skate_ug.MdaPotenciaUnitaria.count())
print(skate_ug_grouped.Pot_UG.sum() - skate_ug.MdaPotenciaUnitaria.sum())
print(bdworgs_ug_grouped.Pot_UG.sum() - bdworgs_ug_grouped.Pot_UG.sum())
print(bdworgs_ug_grouped.Quant_UG.count() - bdworgs_ug_grouped.Quant_UG.count())


0
-1.4901161193847656e-08
0.0
0


### Junta bancos de dados skate_ug e skate_usina pela coluna IdeUsinaOutorga. Todos os atributos de cada usina são passados para cada UG pertecente a usina.

#### Entrada: skate_ug_grouped & skate_usinas
#### Saída: skate_merged

In [29]:
skate_merged  = pd.merge(skate_ug_grouped.drop(["SigTipoGeracao"],axis=1),
        skate_usinas,
        on="IdeUsinaOutorga",
         how="outer")
mask  = skate_merged.DscComercializacaoEnergia == "ACR"
skate_merged.loc[mask,"DatUGInicioOpComerOutorgado"] = skate_merged.DatInicioSuprimento
skate_merged.sample(10)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,Quant_UG,Pot_UG,DatOutorgaUsina,UFUsina,SigTipoGeracao,...,DatConcretagemOutorgado,DatConcretagemRealizado,DatMontagemOutorgado,DatMontagemRealizado,DatEnchimentoOutorgado,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,DatInicioSuprimento,DscComercializacaoEnergia
1144,35146,2020-08-30,2021-01-01,NaN,NaN,1,2334.00,2016-09-27,PR,PCH,...,2019-11-30,None,2020-04-30,None,2020-08-10,None,2019-11-01,None,2021-01-01,ACR
1628,37568,2023-11-30,2023-12-20,NaN,NaN,1,5000.00,2021-12-07,MS,PCH,...,2022-09-01,None,2023-06-01,None,None,None,2023-04-01,None,None,Fora do ACR
3373,49917,2024-07-21,2024-09-21,NaN,NaN,10,42000.00,2022-06-07,BA,EOL,...,2023-11-21,None,2024-04-21,None,None,None,2023-11-21,None,None,Fora do ACR
3135,49206,2024-12-01,2025-01-01,NaN,NaN,29,44100.01,2022-03-08,MG,UFV,...,None,None,2024-05-01,None,None,None,2024-05-01,None,None,Fora do ACR
1390,35884,2021-12-01,2022-01-01,2021-08-17,2021-10-30,9,30933.00,2018-10-18,CE,UFV,...,None,None,2021-02-01,2021-02-15,None,None,2021-09-01,2021-02-15,2022-01-01,ACR
3195,49421,2023-12-09,2024-02-09,NaN,NaN,10,42000.00,2022-02-01,BA,EOL,...,2023-04-09,None,2023-09-09,None,None,None,2023-04-09,None,None,Fora do ACR
2207,40735,2021-12-01,2022-01-01,NaN,NaN,33,49999.95,2020-06-09,PE,UFV,...,None,None,2021-05-01,None,None,None,2021-03-01,None,None,Fora do ACR
1052,34669,2024-09-15,2025-01-01,NaN,NaN,1,6534.00,2020-07-14,SC,PCH,...,2023-11-01,None,2024-05-01,None,2024-09-01,None,2024-04-01,None,2025-01-01,ACR
3732,55998,2022-04-25,2022-05-01,2022-05-18,2022-06-22,1,1273.00,2021-11-05,SP,UTE,...,None,None,2022-02-09,2022-04-10,None,None,2022-01-03,2022-02-03,2022-05-01,ACR
270,31103,2015-03-11,2015-03-20,NaN,NaN,1,2300.00,2013-04-16,RN,EOL,...,2014-04-01,None,2014-06-01,None,None,None,2014-08-15,None,None,Fora do ACR


### Converte colunas cujos nomes iniciam em "Dat" para tipo de data. É necessário que as colunas estejam nomeadas no padrão do skate.

In [22]:
colunas_datas = []
for col in bdworgs_ug_grouped.columns:
    if col[0:3] == "Dat":
        colunas_datas.append(col)
        print(col)
        skate_merged[col] = pd.to_datetime(skate_merged[col],format="%Y-%m-%d",errors="coerce")
        bdworgs_ug_grouped[col] = pd.to_datetime(bdworgs_ug_grouped[col],format="%Y-%m-%d",errors="coerce")

DatInicioOpTesteOutorgada
DatUGInicioOpComerOutorgado
DatLiberacaoSFGTeste
DatLiberOpComerRealizado
DatInicioObraOutorgado
DatInicioObraRealizado
DatConcretagemOutorgado
DatConcretagemRealizado
DatMontagemOutorgado
DatMontagemRealizado
DatEnchimentoOutorgado
DatEnchimentoRealizado
DatSisTransmissaoOutorgado
DatSisTransmissaoRealizado


### Checa quais colunas não estão presentes em cada um dos dataframes

In [23]:
print("Colunas que estão no skate, porém não no bdworgs:")
for col in skate_merged.columns:
    if col not in bdworgs.columns:
        print(col)
        
print("-"*10)
print("Colunas que estão no bdworgs, porém não no skate:")
for col in bdworgs.columns:
    if col not in skate_merged.columns:
        print(col)

Colunas que estão no skate, porém não no bdworgs:
DatOutorgaUsina
DatInicioSuprimento
----------
Colunas que estão no bdworgs, porém não no skate:
Torres_Outorga
Torres_Real


### Junta BDs skate e bdworgs.
#### Entrada: skate_merged & bdworgs_ug_grouped
#### Saída: skate_merged

In [25]:
bdworgs_ug_grouped["origem"] = "bdworgs"
skate_merged["origem"] = "skate"
bd_combined = pd.concat([bdworgs_ug_grouped,skate_merged],axis=0,ignore_index=True)
#bd_combined = bdworgs_ug_grouped.copy(deep=True)
bd_combined["origem"] = bd_combined["origem"].astype("category")
bd_combined.DscComercializacaoEnergia = bd_combined.DscComercializacaoEnergia.astype("category")
bd_combined.sample(10)

,IdeUsinaOutorga,DatInicioOpTesteOutorgada,DatUGInicioOpComerOutorgado,DatLiberacaoSFGTeste,DatLiberOpComerRealizado,UFUsina,VlrPotenciaOutorgada,DatInicioObraOutorgado,DatInicioObraRealizado,DatConcretagemOutorgado,...,DatEnchimentoRealizado,DatSisTransmissaoOutorgado,DatSisTransmissaoRealizado,DscComercializacaoEnergia,SigTipoGeracao,Quant_UG,Pot_UG,origem,DatOutorgaUsina,DatInicioSuprimento
1002,30387,2013-03-01,2013-04-01,2015-03-06,2015-07-23,SC,2.05,2011-03-03,2011-05-03,2011-09-01,...,NaT,2012-09-01,NaT,Fora do ACR,CGH,2,2.05,bdworgs,NaN,NaN
3623,35661,NaT,2022-07-19,NaT,NaT,MS,50000.00,NaT,NaT,NaT,...,NaT,NaT,NaT,Fora do ACR,UFV,16,50000.00,skate,2019-07-02,None
1310,30897,2015-12-01,2015-12-31,2014-10-25,2014-12-09,RN,30.00,2015-05-01,2013-10-30,2015-07-01,...,NaT,2015-03-01,2013-09-30,ACR,EOL,1,3.00,bdworgs,NaN,NaN
2076,33682,2017-11-18,2017-12-31,2017-08-11,2017-09-01,MA,27.60,2017-05-16,2016-10-30,2017-06-16,...,NaT,2017-04-16,2016-06-30,ACR,EOL,3,6.90,bdworgs,NaN,NaN
4752,44513,2023-12-01,2024-01-01,NaT,NaT,BA,50000.00,2023-01-01,NaT,NaT,...,NaT,2023-02-01,NaT,Fora do ACR,UFV,16,50000.00,skate,2021-05-18,None
267,28725,2004-05-01,2004-07-15,2008-02-28,2008-04-15,RS,26.00,2003-03-01,2006-03-29,2003-10-01,...,NaT,NaT,NaT,Fora do ACR,PCH,3,26.00,bdworgs,NaN,NaN
110,27673,NaT,2003-03-17,2010-05-04,2010-06-08,MG,11.00,2001-08-24,2008-08-01,NaT,...,2010-04-16,NaT,NaT,Fora do ACR,PCH,1,11.00,bdworgs,NaN,NaN
5450,49189,2021-12-01,2022-01-01,NaT,NaT,MG,44100.00,2021-03-01,NaT,NaT,...,NaT,2021-05-01,NaT,Fora do ACR,UFV,29,44100.01,skate,2021-02-23,None
1291,30874,2013-12-02,2014-01-01,2015-03-05,2015-05-30,RN,29.70,NaT,2014-02-24,NaT,...,NaT,NaT,2013-01-21,Fora do ACR,EOL,11,29.70,bdworgs,NaN,NaN
5486,49340,NaT,2025-01-18,NaT,NaT,SP,48600.00,NaT,NaT,NaT,...,NaT,NaT,NaT,Fora do ACR,UFV,216,48600.00,skate,2022-01-18,None


In [26]:
# Exporta dataframe
bd_combined.to_parquet("bd_combined.gzip")